In [1]:
import glob
import os
import pickle

from nltk import TweetTokenizer, metrics
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

import utils
import wandb
%load_ext autoreload
%autoreload 2
from utils import read_data_into_DFs, data_for_bert, get_results, create_df, create_df_bert
import numpy as np
import pandas as pd
import sklearn
import torch
import logging
from simpletransformers.classification import ClassificationModel, ClassificationArgs

from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from xml.etree import ElementTree


In [8]:
cwd = os.getcwd()
folder = "../pan21-author-profiling-training-2021-03-14"
DFs = read_data_into_DFs(folder)
os.chdir(cwd)

en/7610534fcf5bf40d6e4b4073f2b980b4.xml
en/b897c69b28b6e80bd4d3fdaf4512e945.xml
en/e8141313febb028ea3c9912f89fcd6b4.xml
en/0d2c0d6880388acbb2efc910c6a7dba2.xml
en/e69115769bae2a02953934dc9c30ae05.xml
en/1946bfa3e2adc568cfdb5480bdf5829d.xml
en/061a343a9faec7d73cedd1aedc59b07c.xml
en/fdd27881da241cff7115c1e89caba6dd.xml
en/b8b56ee954ec519d2beb62e5583f6501.xml
en/3ec0653c7389121735613e3247a8ab6f.xml
en/9946abb814f66c4206d0aefab2e88f69.xml
en/05e1527e1d7210f6da3a90d4c7e53dd7.xml
en/ab8e5fd81fbce9ccf42465e2c5e619f0.xml
en/3348345aac7462125d72b02c3a19fe3d.xml
en/ff40a5a10d32a42946701e7965bf73e3.xml
en/7eca71debd2030ba16d5849a51110df7.xml
en/ab14ad85f29db248a46654fe4bfc2cef.xml
en/e291bd05ee5636bea526fdba1a6aee0c.xml
en/f6dbb398364073aa25cd1aab159a6e71.xml
en/07275a64a6a197f348253cd98bf6ddc5.xml
en/20f4b6fd3cf0e111f6dc244a7cbef7ce.xml
en/35dad41e25a45dcf6d7e3dd905dd0914.xml
en/ba89cd8519b88f7859e7fe117246fb8f.xml
en/876d4f0b18ba27bd45769d7de16fdb3b.xml
en/25b6920ba3dcfabffd201908b1da59a4.xml


ParseError: not well-formed (invalid token): line 65, column 47 (<string>)

In [3]:
xmls_directory = "../../pan21-author-profiling-test-without-gold-for-participants/es"
def _read_xml(author_id):
    tweets = []
    xml_filename = "{}.xml".format(author_id)
    tree = ElementTree.parse(
        os.path.join(xmls_directory, xml_filename),
        parser=ElementTree.XMLParser(encoding="utf-8"),
    )
    root = tree.getroot()
    for child in root[0]:
        tweets.append(child.text)
    return " ".join(tweets)


files = glob.glob("{}/*.xml".format(xmls_directory))
author_ids = list(
    map(lambda x: os.path.splitext(os.path.basename(x))[0], files))
df = pd.DataFrame({"author_id": author_ids})

df["tweet"] = df["author_id"].apply(_read_xml)

ParseError: not well-formed (invalid token): line 65, column 47 (<string>)

In [6]:
fold = [DFs[0][:20], DFs[0][20:40], DFs[0][40:60], DFs[0][60:80], DFs[0][80:100],
        DFs[0][100:120], DFs[0][120:140], DFs[0][140:160], DFs[0][160:180], DFs[0][180:200]]

In [10]:
train = []
dev = []
for i, testi in enumerate(fold):
    train.append(fold[:i]+fold[i+1:])
    dev.append(testi)

In [13]:
for x in range(10):
    DF = train[x][0]
    for i, group in enumerate(train[x]):
        if i == 0:
            continue
        DF = pd.concat([DF, group], axis=0)
    DF = pd.concat([DF, dev[x]])

In [3]:
train_df1, eval_df1 = data_for_bert(DFs[1])



logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = ClassificationArgs()
model_args.num_train_epochs = 50
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.manual_seed = 4
model_args.use_multiprocessing = True
model_args.train_batch_size = 16
model_args.eval_batch_size = 8
model_args.labels_list = [0, 1]
model_args.learning_rate = 5.245514797925528e-05
model_args.wandb_project = "Sweep Results"
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.001
model_args.early_stopping_metric = "mcc"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluate_during_training_steps = 30
model_args.evaluate_during_training_verbose = True
#5.202081638426634e-05

In [5]:
arg_values = []

pd.set_option('display.max_rows', 80)

# For all of the arguments...
"""for arg in dir(model.args):

    # Skip over the special attributes and any functions.
    if (not arg[0:2] == '__') and (not callable(getattr(model.args, arg))):

        # Store the argument and its value as a tuple.
        arg_values.append((arg, str(getattr(model.args, arg))))

# Store as a dataframe just to get the pretty printout.
df_args = pd.DataFrame(arg_values)

df_args"""

,0,1
0,adafactor_beta1,None
1,adafactor_clip_threshold,1.0
2,adafactor_decay_rate,-0.8
3,adafactor_eps,"(1e-30, 0.001)"
4,adafactor_relative_step,True
...,...,...
88,wandb_kwargs,{}
89,wandb_project,Sweep Results
90,warmup_ratio,0.06
91,warmup_steps,0


In [3]:
vec_params = {
        "word_range": (1, 2),
        "word_min": 0.05,
        "word_max": 0.85,
        "char_range": (1, 6),
        "char_min": 0.001,
        "char_max": 1.0}
params_en = {
        "C": 22458.78034,
        "class_weight": "balanced",
        "intercept_scaling": 0.877,
        "loss": "hinge",
        "max_iter": 2000,
        "random_state": 42,
        "tol": 0.00014376187,
    }

In [6]:
#Spanish
vec_params = {
        "word_range": (1, 2),
        "word_min": 0.05,
        "word_max": 0.85,
        "char_range": (1, 6),
        "char_min": 0.001,
        "char_max": 1.0}
X_train, X_dev, Y_train, Y_dev = create_df(DFs[1], prepro=True)
X = pd.concat([X_train, X_dev], ignore_index=True)
Y = pd.concat([Y_train, Y_dev], ignore_index=True)

In [7]:
word_Tfidf_vec = TfidfVectorizer(analyzer='word',
                                         tokenizer=TweetTokenizer().tokenize,
                                         ngram_range=vec_params['word_range'],
                                         min_df=vec_params['word_min'], max_df=vec_params["word_max"],
                                         use_idf=True,
                                         sublinear_tf=True)
char_Tfidf_vec = TfidfVectorizer(analyzer='char',
                                 tokenizer=TweetTokenizer().tokenize,
                                 ngram_range=vec_params['char_range'],
                                 min_df=vec_params['char_min'], max_df=vec_params["char_max"],
                                 use_idf=True,
                                 sublinear_tf=True)

column_transformer = ColumnTransformer(
    [('tfidf_word', word_Tfidf_vec, 'concat_1'),
     ('tfidf_char', char_Tfidf_vec, 'concat_2')],
    remainder='passthrough')
X = column_transformer.fit_transform(X)

In [9]:
model = LinearSVC()
class_weight = ["balanced"]
intercept_scaling= [0.1, 0.01]
loss = ["squared_hinge", "hinge"]
max_iter = [2000]
tol = [0.1, 0.0001437]
C = [22000]
# define grid search
grid = dict(C=C, class_weight=class_weight, intercept_scaling=intercept_scaling,
            loss=loss, max_iter=max_iter, tol=tol)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.795000 using {'C': 22000, 'class_weight': 'balanced', 'intercept_scaling': 0.01, 'loss': 'hinge', 'max_iter': 2000, 'tol': 0.1}
0.791667 (0.084738) with: {'C': 22000, 'class_weight': 'balanced', 'intercept_scaling': 0.1, 'loss': 'squared_hinge', 'max_iter': 2000, 'tol': 0.1}
0.790000 (0.085049) with: {'C': 22000, 'class_weight': 'balanced', 'intercept_scaling': 0.1, 'loss': 'squared_hinge', 'max_iter': 2000, 'tol': 0.0001437}
0.793333 (0.084393) with: {'C': 22000, 'class_weight': 'balanced', 'intercept_scaling': 0.1, 'loss': 'hinge', 'max_iter': 2000, 'tol': 0.1}
0.790000 (0.085049) with: {'C': 22000, 'class_weight': 'balanced', 'intercept_scaling': 0.1, 'loss': 'hinge', 'max_iter': 2000, 'tol': 0.0001437}
0.793333 (0.084393) with: {'C': 22000, 'class_weight': 'balanced', 'intercept_scaling': 0.01, 'loss': 'squared_hinge', 'max_iter': 2000, 'tol': 0.1}
0.791667 (0.084738) with: {'C': 22000, 'class_weight': 'balanced', 'intercept_scaling': 0.01, 'loss': 'squared_hinge', 'max_ite

In [3]:
#English
vec_params2 = {
        "word_range": (1, 2),
        "word_min": 0.05,
        "word_max": 0.85,
        "char_range": (1, 6),
        "char_min": 0.001,
        "char_max": 1.0}
X_train2, X_dev2, Y_train2, Y_dev2 = create_df(DFs[0], prepro=True)
X2 = pd.concat([X_train2, X_dev2], ignore_index=True)
Y2 = pd.concat([Y_train2, Y_dev2], ignore_index=True)

In [4]:
word_Tfidf_vec2 = TfidfVectorizer(analyzer='word',
                                         tokenizer=TweetTokenizer().tokenize,
                                         ngram_range=vec_params2['word_range'],
                                         min_df=vec_params2['word_min'], max_df=vec_params2["word_max"],
                                         use_idf=True,
                                         sublinear_tf=True)
char_Tfidf_vec2 = TfidfVectorizer(analyzer='char',
                                 tokenizer=TweetTokenizer().tokenize,
                                 ngram_range=vec_params2['char_range'],
                                 min_df=vec_params2['char_min'], max_df=vec_params2["char_max"],
                                 use_idf=True,
                                 sublinear_tf=True)

column_transformer2 = ColumnTransformer(
    [('tfidf_word', word_Tfidf_vec2, 'concat_1'),
     ('tfidf_char', char_Tfidf_vec2, 'concat_2')],
    remainder='passthrough')
X2 = column_transformer2.fit_transform(X2)

In [5]:
#https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
model = LinearSVC()
class_weight = ["balanced"]
intercept_scaling= [0.1, 0.877]
loss = ["hinge"]
max_iter = [2000]
tol = [0.00001, 0.1, 0.00014376187]
C = [110000, 22000, 100000]
# define grid search
grid = dict(C=C, class_weight=class_weight, intercept_scaling=intercept_scaling,
            max_iter=max_iter, tol=tol)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X2, Y2)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.656667 using {'C': 22000, 'class_weight': 'balanced', 'intercept_scaling': 0.877, 'max_iter': 2000, 'tol': 0.1}
0.641667 (0.118380) with: {'C': 110000, 'class_weight': 'balanced', 'intercept_scaling': 0.1, 'max_iter': 2000, 'tol': 1e-05}
0.636667 (0.121747) with: {'C': 110000, 'class_weight': 'balanced', 'intercept_scaling': 0.1, 'max_iter': 2000, 'tol': 0.1}
0.641667 (0.118380) with: {'C': 110000, 'class_weight': 'balanced', 'intercept_scaling': 0.1, 'max_iter': 2000, 'tol': 0.00014376187}
0.641667 (0.120474) with: {'C': 110000, 'class_weight': 'balanced', 'intercept_scaling': 0.877, 'max_iter': 2000, 'tol': 1e-05}
0.646667 (0.116857) with: {'C': 110000, 'class_weight': 'balanced', 'intercept_scaling': 0.877, 'max_iter': 2000, 'tol': 0.1}
0.641667 (0.120474) with: {'C': 110000, 'class_weight': 'balanced', 'intercept_scaling': 0.877, 'max_iter': 2000, 'tol': 0.00014376187}
0.641667 (0.118380) with: {'C': 22000, 'class_weight': 'balanced', 'intercept_scaling': 0.1, 'max_iter': 2

In [5]:
#English
vec_params2 = {
        "word_range": (1, 2),
        "word_min": 0.1,
        "word_max": 1.0,
        "char_range": (2, 6),
        "char_min": 0.05,
        "char_max": 0.75}
#Spanish
vec_params2 = {
        "word_range": (1, 2),
        "word_min": 0.05,
        "word_max": 0.85,
        "char_range": (1, 6),
        "char_min": 0.001,
        "char_max": 1.0}

In [3]:
text_for_bert_en = data_for_bert(DFs[1], train=False)
berts_opinion_en = utils.get_berts_opinion(text_for_bert_en, "models/bert/es/checkpoint-396-epoch-2")

  0%|          | 0/200 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3074 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/1049 [00:00<?, ?it/s]

In [6]:
X_train2, X_dev2, Y_train2, Y_dev2 = create_df_bert(DFs[1], berts_opinion_en, prepro=True)
X2 = pd.concat([X_train2, X_dev2], ignore_index=True)
Y2 = pd.concat([Y_train2, Y_dev2], ignore_index=True)

In [7]:
word_Tfidf_vec2 = TfidfVectorizer(analyzer='word',
                                         tokenizer=TweetTokenizer().tokenize,
                                         ngram_range=vec_params2['word_range'],
                                         min_df=vec_params2['word_min'], max_df=vec_params2["word_max"],
                                         use_idf=True,
                                         sublinear_tf=True)
char_Tfidf_vec2 = TfidfVectorizer(analyzer='char',
                                 tokenizer=TweetTokenizer().tokenize,
                                 ngram_range=vec_params2['char_range'],
                                 min_df=vec_params2['char_min'], max_df=vec_params2["char_max"],
                                 use_idf=True,
                                 sublinear_tf=True)

column_transformer2 = ColumnTransformer(
    [('tfidf_word', word_Tfidf_vec2, 'concat_1'),
     ('tfidf_char', char_Tfidf_vec2, 'concat_2')],
    remainder='passthrough')
X2 = column_transformer2.fit_transform(X2)

In [9]:
#https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
model = LinearSVC()
class_weight = [None]
intercept_scaling= [0.1, 0.01, 1, 5]
loss = ["squared_hinge", "hinge"]
max_iter = [2000]
tol = [0.1, 0.0001, 0.00014376187, 1.7355634213475965e-05]
C = [0.35, 200000, 5000, 8700]
# define grid search
grid = dict(C=C, class_weight=class_weight, intercept_scaling=intercept_scaling, loss=loss,
            max_iter=max_iter, tol=tol)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X2, Y2)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.960000 using {'C': 0.35, 'class_weight': None, 'intercept_scaling': 0.1, 'loss': 'squared_hinge', 'max_iter': 2000, 'tol': 0.1}
0.960000 (0.032660) with: {'C': 0.35, 'class_weight': None, 'intercept_scaling': 0.1, 'loss': 'squared_hinge', 'max_iter': 2000, 'tol': 0.1}
0.960000 (0.032660) with: {'C': 0.35, 'class_weight': None, 'intercept_scaling': 0.1, 'loss': 'squared_hinge', 'max_iter': 2000, 'tol': 0.0001}
0.960000 (0.032660) with: {'C': 0.35, 'class_weight': None, 'intercept_scaling': 0.1, 'loss': 'squared_hinge', 'max_iter': 2000, 'tol': 0.00014376187}
0.960000 (0.032660) with: {'C': 0.35, 'class_weight': None, 'intercept_scaling': 0.1, 'loss': 'squared_hinge', 'max_iter': 2000, 'tol': 1.7355634213475965e-05}
0.960000 (0.032660) with: {'C': 0.35, 'class_weight': None, 'intercept_scaling': 0.1, 'loss': 'hinge', 'max_iter': 2000, 'tol': 0.1}
0.960000 (0.032660) with: {'C': 0.35, 'class_weight': None, 'intercept_scaling': 0.1, 'loss': 'hinge', 'max_iter': 2000, 'tol': 0.0001}

In [13]:
cwd = os.getcwd()
folder = "../pan21-author-profiling-training-2021-03-14"
DFs = read_data_into_DFs(folder)
os.chdir(cwd)
vec_params = {
    "word_range": (1, 2),
    "word_min": 0.1,
    "word_max": 1.0,
    "char_range": (2, 6),
    "char_min": 0.05,
    "char_max": 0.75}
model_params = {
        "C": 0.35,
        "class_weight": None,
        "intercept_scaling": 0.1,
        "loss": "squared_hinge",
        "max_iter": 2000,
        "random_state": 42,
        "tol": 0.1,}
data_bert = data_for_bert(DFs[0], train=False)
berts_opinion, bert_results = utils.get_berts_opinion(data_bert, "models/bert/en/checkpoint-396-epoch-2")
#mylist = data_for_bert(DFs[0], train=False)
#mylist = mylist.values.tolist()
#berts_opinion = get_berts_opinion(mylist, "models/bert/en/best_model")

  0%|          | 0/200 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3719 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/988 [00:00<?, ?it/s]

In [14]:
X_train, X_dev, Y_train, Y_dev = create_df_bert(DFs[0], berts_opinion)

model = LinearSVC(**model_params)

word_Tfidf_vec = TfidfVectorizer(analyzer='word',
                                     tokenizer=TweetTokenizer().tokenize,
                                     ngram_range=vec_params['word_range'],
                                     min_df=vec_params['word_min'], max_df=vec_params["word_max"],
                                     use_idf=True,
                                     sublinear_tf=True)
char_Tfidf_vec = TfidfVectorizer(analyzer='char',
                                 tokenizer=TweetTokenizer().tokenize,
                                 ngram_range=vec_params['char_range'],
                                 min_df=vec_params['char_min'], max_df=vec_params["char_max"],
                                 use_idf=True,
                                 sublinear_tf=True)

column_transformer = ColumnTransformer(
    [('tfidf_word', word_Tfidf_vec, 'concat_1'),
     ('tfidf_char', char_Tfidf_vec, 'concat_2')],
    remainder='passthrough')

pipe = Pipeline([('tfidf', column_transformer),
                 ('classify', model)])

pipe.fit(X_train, Y_train)
X_dev = column_transformer.transform(X_dev)

predictions = model.predsklearn ict(X_dev)

accuracy, f_measures = get_results(Y_dev, predictions, "es")

Language: es
Accuracy: 0.7
Precisions: 0.68, 0.7333333333333333
Recalls: 0.8095238095238095, 0.5789473684210527
F Scores: 0.7391304347826089, 0.6470588235294117


In [17]:
Y = pd.concat([Y_train, Y_dev], ignore_index=True)
Y = [int(y) for y in Y]
accuracy_bert, f_measures_bert = get_results(Y, bert_results, "es")


Language: es
Accuracy: 0.825
Precisions: 0.7443609022556391, 0.9850746268656716
Recalls: 0.99, 0.66
F Scores: 0.8497854077253219, 0.7904191616766467


In [5]:
vec_params = {
        "word_range": (1, 2),
        "word_min": 0.05,
        "word_max": 0.85,
        "char_range": (1, 6),
        "char_min": 0.001,
        "char_max": 1.0}
model_params = {
        "C": 100000,
        "class_weight": "balanced",
        "intercept_scaling": 0.1,
        #"loss": "squared_hinge",
        "max_iter": 2000,
        "random_state": 42,
        "tol": 0.00001}

X_train, X_dev, Y_train, Y_dev = create_df(DFs[1], prepro=True, demoji=False)

model = LogisticRegression(**model_params)

word_Tfidf_vec = TfidfVectorizer(analyzer='word',
                                     tokenizer=TweetTokenizer().tokenize,
                                     ngram_range=vec_params['word_range'],
                                     min_df=vec_params['word_min'], max_df=vec_params["word_max"],
                                     use_idf=True,
                                     sublinear_tf=True)
char_Tfidf_vec = TfidfVectorizer(analyzer='char',
                                 tokenizer=TweetTokenizer().tokenize,
                                 ngram_range=vec_params['char_range'],
                                 min_df=vec_params['char_min'], max_df=vec_params["char_max"],
                                 use_idf=True,
                                 sublinear_tf=True)

column_transformer = ColumnTransformer(
    [('tfidf_word', word_Tfidf_vec, 'concat_1'),
     ('tfidf_char', char_Tfidf_vec, 'concat_2')],
    remainder='passthrough')

pipe = Pipeline([('tfidf', column_transformer),
                 ('classify', model)])

pipe.fit(X_train, Y_train)
X_dev = column_transformer.transform(X_dev)
X_train = column_transformer.transform(X_train)



probs_train = model.predict_proba(X_train)
probs_dev = model.predict_proba(X_dev)

In [6]:
preds = model.predict(X_dev)
acc = sklearn.metrics.accuracy_score(Y_dev, preds)

In [7]:
#"models/bert/en/checkpoint-396-epoch-2"
data_bert = data_for_bert(DFs[1], train=False)
berts_opinion = utils.get_berts_opinion(data_bert, "models/bert/es")

  0%|          | 0/200 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3068 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/2102 [00:00<?, ?it/s]

In [8]:
X_train_combo = np.concatenate([probs_train, berts_opinion[:160]], axis=1)
X_dev_combo = np.concatenate([probs_dev, berts_opinion[160:]], axis=1)

In [14]:
model_params_svm = {
        "C": 0.017,
        "class_weight": None,
        "intercept_scaling": 5,
        "loss": "hinge",
        "max_iter": 2000,
        "random_state": 42,
        "tol": 5}
model_svm = LinearSVC(**model_params_svm)

model_svm.fit(X_train_combo, Y_train)

predictions = model_svm.predict(X_dev_combo)

accuracy = sklearn.metrics.accuracy_score(Y_dev, predictions)
get_results(Y_dev, predictions, "en")

Language: en
Accuracy: 0.775
Precisions: 0.8, 0.75
Recalls: 0.7619047619047619, 0.7894736842105263
F Scores: 0.7804878048780488, 0.7692307692307692


[0.775, array([0.7804878 , 0.76923077])]

In [10]:
class_weight = [None, "balanced"]
intercept_scaling= [0.5, 0.05, 1, 5, 0.005, 0.0005, 0.1]
loss = ["hinge", "squared_hinge"]
max_iter = [2000]
tol = [0.00001, 5, 0.0005, 0.00015]
C = [0.015, 0.017, 0.01, 0.02]
# define grid search
grid = dict(C=C, class_weight=class_weight, intercept_scaling=intercept_scaling, loss=loss,
            max_iter=max_iter, tol=tol)

In [11]:
best_acc = 0
for g in sklearn.model_selection.ParameterGrid(grid):
    model = LinearSVC(**g)
    model.fit(X_train_combo, Y_train)
    predictions = model.predict(X_dev_combo)
    accuracy = sklearn.metrics.accuracy_score(Y_dev, predictions)
    print(f"Accuracy: {accuracy}, {g}")
    if accuracy > best_acc:
        best_acc = accuracy
        best_grid = g

Accuracy: 0.775, {'C': 0.015, 'class_weight': None, 'intercept_scaling': 0.5, 'loss': 'hinge', 'max_iter': 2000, 'tol': 1e-05}
Accuracy: 0.775, {'C': 0.015, 'class_weight': None, 'intercept_scaling': 0.5, 'loss': 'hinge', 'max_iter': 2000, 'tol': 5}
Accuracy: 0.775, {'C': 0.015, 'class_weight': None, 'intercept_scaling': 0.5, 'loss': 'hinge', 'max_iter': 2000, 'tol': 0.0005}
Accuracy: 0.775, {'C': 0.015, 'class_weight': None, 'intercept_scaling': 0.5, 'loss': 'hinge', 'max_iter': 2000, 'tol': 0.00015}
Accuracy: 0.775, {'C': 0.015, 'class_weight': None, 'intercept_scaling': 0.5, 'loss': 'squared_hinge', 'max_iter': 2000, 'tol': 1e-05}
Accuracy: 0.775, {'C': 0.015, 'class_weight': None, 'intercept_scaling': 0.5, 'loss': 'squared_hinge', 'max_iter': 2000, 'tol': 5}
Accuracy: 0.775, {'C': 0.015, 'class_weight': None, 'intercept_scaling': 0.5, 'loss': 'squared_hinge', 'max_iter': 2000, 'tol': 0.0005}
Accuracy: 0.775, {'C': 0.015, 'class_weight': None, 'intercept_scaling': 0.5, 'loss': 'squa